In [6]:
from ultralytics.nn.modules import MCDropout
#from ultralytics.data.dataset import YOLODataset
from torch.utils.data import DataLoader
from ultralytics import YOLO
import torch
import torch.nn.functional as F
import torch.nn as nn
#from ultralytics.utils.loss import v8DetectionLoss
from torchvision.ops import box_iou
from torch.optim import Adam
from torch.optim import AdamW
from torch.optim import RMSprop
from torch.optim import SGD
from torch.optim import Rprop
from torch.optim import NAdam
from torch.optim import Adam
import numpy as np
from ultralytics.data.dataset import YOLODataset
from ultralytics.utils.ops import non_max_suppression
import matplotlib.pyplot as plt

In [7]:
from loaddata import create_validation_loader

ImportError: cannot import name 'create_validation_loader' from 'loaddata' (/home/omid/loaddata.py)

## implemntation of Dropout

In [ ]:


class YOLOWithDropout(nn.Module):

    def __init__(self , original_model , n_samples = 10):
        super(YOLOWithDropout, self).__init__()
        self.model = original_model.model
        
        self.n_samples = n_samples


        
    

    def enable_dropout(self):
            """Activate dropout during inference while keeping batch norm in eval mode."""
            self.model.eval()
            for module in self.model.modules():
                if isinstance(module, MCDropout):
                    module.train()
            return True
            
        


    def evaluate_dropout(self, val_loader):
        
        

        all_logits_standard = []
        all_labels_standard = []
        


        with torch.no_grad():

            for batch in val_loader:

                input = batch['img'].to(torch.float32) / 255.0  # Normalize input to [0, 1] range
                input = input 
                targets = {
                    'cls' : batch['cls'] , 
                    'bboxes' : batch['bboxes'] , 
                    'batch_idx' : batch['batch_idx']
                }

                outputs = self.model(input)
                outputs = outputs[0][0].transpose(-1, -2)
                #print('outputs shape is for normal ',outputs.shape)
                logits  , labels = self._match_targets_to_outputs(outputs , targets)
                if logits is not None and labels is not None:
                    all_logits_standard.append(logits)
                    all_labels_standard.append(labels)
                    
                    
        
        all_logits_standard = torch.cat(all_logits_standard , dim = 0)
        all_labels_standard = torch.cat(all_labels_standard , dim = 0).long()
        

        ece = _ECE_criterion()
        nll_criterion = nn.CrossEntropyLoss().cuda()

        ece_standard = ece(all_logits_standard , all_labels_standard.squeeze(1))
        nll_standard = nll_criterion(all_logits_standard , all_labels_standard.squeeze(1))
        
        print(f'ECE standard is : {ece_standard}')
        print(f'NLL standard is : {nll_standard}')

        # Enable dropout for Monte Carlo sampling
        self.enable_dropout()
             
        all_logits_mc = []
        all_labels_mc = []
        
        for i,batch in enumerate(val_loader):
            input = batch['img'].to(torch.float32)  / 255.0  # Normalize input to [0, 1] range

            print('input shape is ',input.shape)

            targets = {
                    'cls': batch['cls'],
                    'bboxes': batch['bboxes'],
                    'batch_idx': batch['batch_idx']
                }

            outputs = []
            # Collect outputs from multiple forward passes
            
            for _ in range(self.n_samples):
                output = self.model(input)
                output = output[0][0].transpose(-1, -2)
                
                
                outputs.append(output)

            outputs = torch.stack(outputs , dim = 0 )
            mean_outputs = torch.mean(outputs , dim = 0)


            


          


            
            logits,  labels = self._match_targets_to_outputs(mean_outputs, targets)
            
            if logits is not None and labels is not None:
                
                all_logits_mc.append(logits)
                all_labels_mc.append(labels)

        if not all_logits_mc:
            print('no valid predcition found for MC sampling')
            return None

        
        all_logits = torch.cat(all_logits_mc , dim = 0)
        all_labels = torch.cat(all_labels_mc , dim = 0).long()

        
        
        nll_criterion = nn.CrossEntropyLoss().cuda()
        ece_criterion = _ECE_criterion().cuda()
        
        ece1 = ece_criterion(all_logits , all_labels.squeeze(1))
        nll1 = nll_criterion(all_logits , all_labels.squeeze(1))
        print(f'ECE after MC sampling is : {ece1}')
        print(f'NLL after MC sampling is : {nll1}')
        print(f'improvement in ECE is : {((ece_standard - ece1)/ece_standard)*100}')

        

        return ece_standard , ece


        
            


    def _match_targets_to_outputs(self, outputs, targets):  # for each image 
        
        labels = []
        logits  = []
    

        logits_for_nms = outputs.clone()  # Clone to avoid modifying original logits
        logits_for_nms = logits_for_nms.unsqueeze(0).transpose(-1, -2)  # Add batch dimension and transpose to match expected shape
        confidence = torch.sigmoid(logits_for_nms[: , 4:14 , :]  )
        bbox = logits_for_nms[: , 0:4 , :]   # Extract bounding box coordinates (x, y, x, y)
        combined = torch.cat((bbox , confidence) , dim = 1)  # Remove batch dimension if present

        nms_detections = non_max_suppression(combined, conf_thres=0.25,nc = 10, iou_thres=0.45, classes=None, agnostic=False, max_det=300 , in_place=False , return_idxs=True)

      

        original_indices = nms_detections[1][0] # Get the original indices of detections before NMS
        

        if len(nms_detections[0][0]) == 0:
            return None, None
            
        pred_bboxes_xyxy = nms_detections[0][0][..., :4]  # Extract bounding boxes (x, y,x, y)
        targets_bboxes = targets['bboxes']   #[N , 4]

        targets_bboxes_xyxy = torch.cat([targets_bboxes[: , :2]- targets_bboxes[: , 2:4]/2 , targets_bboxes[: , :2]+ targets_bboxes[: , 2:4]/2], dim = 1)
        
        image_height, image_width = 640, 640  # Replace with your image size
        # Normalize bounding boxes to [0, 1] range
        pred_bboxes_xyxy[:, 0] /= image_width   # Normalize x
        pred_bboxes_xyxy[:, 1] /= image_height  # Normalize y
        
        pred_bboxes_xyxy[:, 2] /= image_width   # Normalize x2
        pred_bboxes_xyxy[:, 3] /= image_height  # Normalize y2
        
      
        

        iou_matrix = box_iou(targets_bboxes_xyxy, pred_bboxes_xyxy)   # N , M
        
       
        
        max_iou, max_iou_index = torch.max(iou_matrix , dim = 1)   # shape (N , )  # which detection index in nms has the maximum iou with each target
        
        
        
        for i in range(len(max_iou_index)):
            if iou_matrix[i , max_iou_index[i]] > 0.60:
               
                labels.append(targets['cls'][i])
                
                 # Use the original index to get logits from the original outputs
                original_detection_idx = original_indices[max_iou_index[i]]
                logits.append(outputs[original_detection_idx , 4:14])

            

        print(f"Number of matched logits: {len(logits)}")
        print(f"Number of matched labels: {len(labels)}")
        if len(logits) == 0:
            
            return None, None 
        

        logits= torch.stack(logits , dim = 0) if len(logits)>0 else None
        labels= torch.stack(labels , dim = 0) if len(labels)>0 else None        
        
            
        return logits  , labels

          




model = YOLO('best-with-dropout-layer.pt')

#model = add_dropout_to_model( model,p=0.1)

#dropout_model = YOLOWithDropout(model , n_samples = 5)

#found = dropout_model.enable_dropout()
#print('found is : ',found)




In [ ]:
dropout_model = YOLOWithDropout(model, n_samples=5)



#dropout_model.evaluate_dropout(valid_loader)
dropout_model.evaluate_dropout(valid_loader)